In [ ]:
import cv2
import numpy as np
import random
from pathlib import Path

def rotate_image_and_yolo_labels(image_path, label_path, angle):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    
    angle_rad = np.radians(angle)
    cos_val = abs(np.cos(angle_rad))
    sin_val = abs(np.sin(angle_rad))
    new_w = int(h * sin_val + w * cos_val)
    new_h = int(h * cos_val + w * sin_val)
    
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    M[0, 2] += (new_w - w) / 2
    M[1, 2] += (new_h - h) / 2
    
    rotated_image = cv2.warpAffine(image, M, (new_w, new_h), 
                                    borderMode=cv2.BORDER_CONSTANT, 
                                    borderValue=(0, 0, 0))
    
    with open(label_path, 'r') as f:
        labels = [line.strip().split() for line in f.readlines()]
    
    new_labels = []
    for label in labels:
        class_id = label[0]
        x_center, y_center, width, height = map(float, label[1:5])
        
        x_center_px = x_center * w
        y_center_px = y_center * h
        box_w_px = width * w
        box_h_px = height * h
        
        corners = np.array([
            [x_center_px - box_w_px/2, y_center_px - box_h_px/2, 1],
            [x_center_px + box_w_px/2, y_center_px - box_h_px/2, 1],
            [x_center_px + box_w_px/2, y_center_px + box_h_px/2, 1],
            [x_center_px - box_w_px/2, y_center_px + box_h_px/2, 1]
        ])
        
        rotated_corners = (M @ corners.T).T
        
        x_coords = rotated_corners[:, 0]
        y_coords = rotated_corners[:, 1]
        
        x_min, x_max = np.min(x_coords), np.max(x_coords)
        y_min, y_max = np.min(y_coords), np.max(y_coords)
        
        x_min = max(0, x_min)
        y_min = max(0, y_min)
        x_max = min(new_w, x_max)
        y_max = min(new_h, y_max)
        
        new_x_center = ((x_min + x_max) / 2) / new_w
        new_y_center = ((y_min + y_max) / 2) / new_h
        new_width = (x_max - x_min) / new_w
        new_height = (y_max - y_min) / new_h
        
        if new_width > 0.01 and new_height > 0.01:
            new_labels.append(f"{class_id} {new_x_center:.6f} {new_y_center:.6f} {new_width:.6f} {new_height:.6f}")
    
    return rotated_image, new_labels

def augment_dataset(images_dir, labels_dir, output_images_dir, output_labels_dir, min_angle=1, max_angle=15, num_augments=1):

    Path(output_images_dir).mkdir(parents=True, exist_ok=True)
    Path(output_labels_dir).mkdir(parents=True, exist_ok=True)
    
    image_files = list(Path(images_dir).glob('*.jpg')) + \
                  list(Path(images_dir).glob('*.png')) + \
                  list(Path(images_dir).glob('*.jpeg'))
    
    for img_file in image_files:
        label_file = Path(labels_dir) / f"{img_file.stem}.txt"
        
        if not label_file.exists():
            print(f"Label tidak ditemukan untuk {img_file.name}")
            continue
        
        for i in range(num_augments):
            angle = random.uniform(min_angle, max_angle)
            if random.random() > 0.5:
                angle = -angle 
            
            rotated_img, rotated_labels = rotate_image_and_yolo_labels(
                str(img_file), str(label_file), angle
            )
            
            output_img_name = f"{img_file.stem}_aug{i}_rot{abs(angle):.1f}.jpg"
            output_img_path = Path(output_images_dir) / output_img_name
            cv2.imwrite(str(output_img_path), rotated_img)
            
            output_label_name = f"{img_file.stem}_aug{i}_rot{abs(angle):.1f}.txt"
            output_label_path = Path(output_labels_dir) / output_label_name
            with open(output_label_path, 'w') as f:
                f.write('\n'.join(rotated_labels))

if __name__ == "__main__":
    augment_dataset(
        images_dir='no_helmet/no_helmet',
        labels_dir='no_helmet',
        output_images_dir='dataset_augmented/tmp/images',
        output_labels_dir='dataset_augmented/tmp/labels',
        min_angle=1,
        max_angle=15,
        num_augments=3 
    )

In [ ]:
import cv2
import numpy as np
from pathlib import Path
import random

def draw_yolo_bbox(image, label_path, color=(0, 255, 0), thickness=2):
    h, w = image.shape[:2]
    
    with open(label_path, 'r') as f:
        labels = [line.strip().split() for line in f.readlines()]
    
    for label in labels:
        class_id = int(label[0])
        x_center, y_center, width, height = map(float, label[1:5])
        
        x_center_px = int(x_center * w)
        y_center_px = int(y_center * h)
        box_w = int(width * w)
        box_h = int(height * h)
        
        x1 = int(x_center_px - box_w / 2)
        y1 = int(y_center_px - box_h / 2)
        x2 = int(x_center_px + box_w / 2)
        y2 = int(y_center_px + box_h / 2)
        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
        
        label_text = f"Class {class_id}"
        cv2.putText(image, label_text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return image

def visualize_augmented_results(images_dir, labels_dir, num_samples=5):
    image_files = list(Path(images_dir).glob('*.jpg')) + \
                  list(Path(images_dir).glob('*.png'))
    
    samples = random.sample(image_files, min(num_samples, len(image_files)))
    
    for img_file in samples:
        label_file = Path(labels_dir) / f"{img_file.stem}.txt"
        
        if not label_file.exists():
            print(f"Label tidak ditemukan: {label_file}")
            continue
        
        image = cv2.imread(str(img_file))
        
        image_with_bbox = draw_yolo_bbox(image.copy(), str(label_file))
        
        window_name = f"{img_file.name}"
        cv2.imshow(window_name, image_with_bbox)
        
        print(f"Menampilkan: {img_file.name}")
        print("Tekan sembarang tombol untuk gambar berikutnya, 'q' untuk keluar")
        
        key = cv2.waitKey(0)
        cv2.destroyWindow(window_name)
        
        if key == ord('q'):
            break
    
    cv2.destroyAllWindows()

def compare_before_after(original_img_path, original_label_path, 
                         augmented_img_path, augmented_label_path):
    img_original = cv2.imread(original_img_path)
    img_original = draw_yolo_bbox(img_original.copy(), original_label_path, 
                                   color=(0, 255, 0))
    
    img_augmented = cv2.imread(augmented_img_path)
    img_augmented = draw_yolo_bbox(img_augmented.copy(), augmented_label_path, 
                                    color=(0, 0, 255))
    
    h1, w1 = img_original.shape[:2]
    h2, w2 = img_augmented.shape[:2]
    
    if h1 != h2:
        target_h = max(h1, h2)
        img_original = cv2.resize(img_original, (int(w1 * target_h / h1), target_h))
        img_augmented = cv2.resize(img_augmented, (int(w2 * target_h / h2), target_h))
    
    comparison = np.hstack([img_original, img_augmented])
    
    cv2.putText(comparison, "ORIGINAL", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(comparison, "AUGMENTED", (w1 + 50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.imshow("Comparison: Original vs Augmented", comparison)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

visualize_augmented_results(
    images_dir='dataset_augmented/images/train',
    labels_dir='dataset_augmented/labels/train',
    num_samples=5
)

In [ ]:
import cv2
import numpy as np
import random
from pathlib import Path

def horizontal_flip_image_and_labels(image_path, label_path):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    
    flipped_image = cv2.flip(image, 1) 
    
    with open(label_path, 'r') as f:
        labels = [line.strip().split() for line in f.readlines()]
    
    new_labels = []
    for label in labels:
        class_id = label[0]
        x_center, y_center, width, height = map(float, label[1:5])
        
        new_x_center = 1.0 - x_center
        
        new_labels.append(f"{class_id} {new_x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    
    return flipped_image, new_labels

def translate_image_and_labels(image_path, label_path, tx_ratio=0.1, ty_ratio=0.1):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    
    tx = int(tx_ratio * w)
    ty = int(ty_ratio * h)
    
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    
    translated_image = cv2.warpAffine(image, M, (w, h), 
                                       borderMode=cv2.BORDER_CONSTANT,
                                       borderValue=(0, 0, 0))
    
    with open(label_path, 'r') as f:
        labels = [line.strip().split() for line in f.readlines()]
    
    new_labels = []
    for label in labels:
        class_id = label[0]
        x_center, y_center, width, height = map(float, label[1:5])
        
        x_center_px = x_center * w
        y_center_px = y_center * h
        box_w = width * w
        box_h = height * h
        
        new_x_center_px = x_center_px + tx
        new_y_center_px = y_center_px + ty
        
        x1 = new_x_center_px - box_w / 2
        y1 = new_y_center_px - box_h / 2
        x2 = new_x_center_px + box_w / 2
        y2 = new_y_center_px + box_h / 2
        
        x1_clipped = max(0, x1)
        y1_clipped = max(0, y1)
        x2_clipped = min(w, x2)
        y2_clipped = min(h, y2)
        
        visible_area = (x2_clipped - x1_clipped) * (y2_clipped - y1_clipped)
        original_area = box_w * box_h
        
        if visible_area < 0.5 * original_area:
            continue
        
        new_x_center_px = (x1_clipped + x2_clipped) / 2
        new_y_center_px = (y1_clipped + y2_clipped) / 2
        new_box_w = x2_clipped - x1_clipped
        new_box_h = y2_clipped - y1_clipped
        
        new_x_center = new_x_center_px / w
        new_y_center = new_y_center_px / h
        new_width = new_box_w / w
        new_height = new_box_h / h
        
        if new_width > 0.01 and new_height > 0.01:
            new_labels.append(f"{class_id} {new_x_center:.6f} {new_y_center:.6f} {new_width:.6f} {new_height:.6f}")
    
    return translated_image, new_labels

def augment_dataset_all(images_dir, labels_dir, output_images_dir, output_labels_dir,
                        do_flip=True, do_translate=True, do_rotate=True,
                        rotate_range=(1, 15), translate_range=(-0.1, 0.1)):
   
    Path(output_images_dir).mkdir(parents=True, exist_ok=True)
    Path(output_labels_dir).mkdir(parents=True, exist_ok=True)
    
    image_files = list(Path(images_dir).glob('*.jpg')) + \
                  list(Path(images_dir).glob('*.png')) + \
                  list(Path(images_dir).glob('*.jpeg'))
    
    for img_file in image_files:
        label_file = Path(labels_dir) / f"{img_file.stem}.txt"
        
        if not label_file.exists():
            print(f"Label tidak ditemukan untuk {img_file.name}")
            continue
        
        aug_count = 0
        
        if do_flip:
            flipped_img, flipped_labels = horizontal_flip_image_and_labels(
                str(img_file), str(label_file)
            )
            
            output_img_name = f"{img_file.stem}_flip.jpg"
            output_img_path = Path(output_images_dir) / output_img_name
            cv2.imwrite(str(output_img_path), flipped_img)
            
            output_label_name = f"{img_file.stem}_flip.txt"
            output_label_path = Path(output_labels_dir) / output_label_name
            with open(output_label_path, 'w') as f:
                f.write('\n'.join(flipped_labels))
            
            aug_count += 1
        
        if do_translate:
            tx_ratio = random.uniform(translate_range[0], translate_range[1])
            ty_ratio = random.uniform(translate_range[0], translate_range[1])
            
            translated_img, translated_labels = translate_image_and_labels(
                str(img_file), str(label_file), tx_ratio, ty_ratio
            )
            
            output_img_name = f"{img_file.stem}_trans_x{tx_ratio:.2f}_y{ty_ratio:.2f}.jpg"
            output_img_path = Path(output_images_dir) / output_img_name
            cv2.imwrite(str(output_img_path), translated_img)
            
            output_label_name = f"{img_file.stem}_trans_x{tx_ratio:.2f}_y{ty_ratio:.2f}.txt"
            output_label_path = Path(output_labels_dir) / output_label_name
            with open(output_label_path, 'w') as f:
                f.write('\n'.join(translated_labels))
            
            aug_count += 1
        
        if do_rotate:
            from rotate_augmentation import rotate_image_and_yolo_labels
            
            angle = random.uniform(rotate_range[0], rotate_range[1])
            if random.random() > 0.5:
                angle = -angle
            
            rotated_img, rotated_labels = rotate_image_and_yolo_labels(
                str(img_file), str(label_file), angle
            )
            
            output_img_name = f"{img_file.stem}_rot{abs(angle):.1f}.jpg"
            output_img_path = Path(output_images_dir) / output_img_name
            cv2.imwrite(str(output_img_path), rotated_img)
            
            output_label_name = f"{img_file.stem}_rot{abs(angle):.1f}.txt"
            output_label_path = Path(output_labels_dir) / output_label_name
            with open(output_label_path, 'w') as f:
                f.write('\n'.join(rotated_labels))
            
            aug_count += 1
        
        if do_flip and do_translate:
            import tempfile

            flipped_img, flipped_labels = horizontal_flip_image_and_labels(
                str(img_file), str(label_file)
            )

            with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_img_file, \
                 tempfile.NamedTemporaryFile(suffix=".txt", mode='w', delete=False) as temp_label_file:
                temp_img_path = temp_img_file.name
                temp_label_path = temp_label_file.name
                cv2.imwrite(temp_img_path, flipped_img)
                temp_label_file.write('\n'.join(flipped_labels))

            tx_ratio = random.uniform(translate_range[0], translate_range[1])
            ty_ratio = random.uniform(translate_range[0], translate_range[1])

            combo_img, combo_labels = translate_image_and_labels(
                temp_img_path, temp_label_path, tx_ratio, ty_ratio
            )

            output_img_name = f"{img_file.stem}_flip_trans.jpg"
            output_img_path = Path(output_images_dir) / output_img_name
            cv2.imwrite(str(output_img_path), combo_img)

            output_label_name = f"{img_file.stem}_flip_trans.txt"
            output_label_path = Path(output_labels_dir) / output_label_name
            with open(output_label_path, 'w') as f:
                f.write('\n'.join(combo_labels))

            import os
            os.remove(temp_img_path)
            os.remove(temp_label_path)

            aug_count += 1
        
        print(f"Selesai: {img_file.name} -> {aug_count} augmentasi")

augment_dataset_all(
    images_dir='no_helmet/no_helmet',
    labels_dir='labels',
    output_images_dir='dataset_augmented/tmp/images',
    output_labels_dir='dataset_augmented/tmp/labels',
    do_flip=True,
    do_translate=True,
    do_rotate=False,
    rotate_range=(1, 15),
    translate_range=(-0.1, 0.1)
)